# Fouille données

In [ ]:
!pip install pandas numpy matplotlib seaborn folium scikit-learn

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.preprocessing import StandardScaler
from collections import Counter

%matplotlib inline
sns.set(style="whitegrid")

# Importation des données

In [ ]:
df = pd.read_csv('flickr_data2.csv', on_bad_lines='skip', sep=",")
print("Nombre de photos au départ :  " + str(len(df)))

df.columns = df.columns.str.strip()

df_clean = df.dropna(axis=1, how='all')

cols_date = ['date_taken_year', 'date_taken_month', 'date_taken_day', 'date_taken_hour', 'date_taken_minute']

df_clean = df_clean.drop_duplicates(subset=['user', 'lat', 'long'] + cols_date)
df_clean = df_clean.dropna(subset=cols_date)

for col in cols_date:
    df_clean[col] = df_clean[col].astype(int)

colonnes_utiles = ['id', 'user', 'lat', 'long', 'tags', 'title'] + cols_date
df_clean = df_clean[colonnes_utiles]

print("\nDonnées nettoyées :")
display(df_clean.head())
print("Nombre de photos restantes : " + str(len(df_clean)))

pourcentage_supprimes = ((len(df) - len(df_clean)) / len(df)) * 100
print(f"Pourcentage de données supprimées : {pourcentage_supprimes:.2f}%")

# Nettoyage des données

In [ ]:
print(f"Année min : {df_clean['date_taken_year'].min()}")
print(f"Année max : {df_clean['date_taken_year'].max()}")

df_clean = df_clean[
    (df_clean['date_taken_year'] >= 2004) &
    (df_clean['date_taken_year'] <= 2026)
]

print(f"Année min : {df_clean['date_taken_year'].min()}")
print(f"Année max : {df_clean['date_taken_year'].max()}")
print("Après filtre temporel : " + str(len(df_clean)) + " photos.")

In [ ]:
# Vérifier que les coordonnées géographiques sont cohérentes.
photos_avant_filtre = len(df_clean)

df_clean = df_clean[
    (df_clean['lat'] >= -90) & (df_clean['lat'] <= 90) &
    (df_clean['long'] >= -180) & (df_clean['long'] <= 180)
]
photos_supprimees = photos_avant_filtre - len(df_clean)
print(f"Photos supprimées par le filtre géographique : {photos_supprimees}")
print(f"Photos avec coordonnées valides : {len(df_clean)}")
print(f"Min/Max Latitude  : {df_clean['lat'].min()} / {df_clean['lat'].max()}")
print(f"Min/Max Longitude : {df_clean['long'].min()} / {df_clean['long'].max()}")

In [ ]:
from folium.plugins import FastMarkerCluster
# Création de la carte centrée sur une latitude et une longitude moyennes
m = folium.Map(location=[45.7640, 4.8357], zoom_start=13)
locations = list(zip(df['lat'], df['long']))
FastMarkerCluster(data=locations).add_to(m)
# Sauvegarde la carte dans un fichier HTML
m.save('ma_carte_lyon.html')
 

# Clustering